# All Cause Mortality
This notebook takes the files from the [WHO Mortality Database](https://www.who.int/data/data-collection-tools/who-mortality-database) and reprocesses them for use on a dashboard.  Only all cause mortality is considered here.  In most cases the all cause mortality is specified by ICD code for a given country.  Three output files are produced that have total mortality numbers, reported population, and mortality per 1000 people.

In [1]:
import pandas as pd
import numpy as np

## Load mortality files

In [2]:
# The following columns for total deaths
deaths_columns = [f'Deaths{i}' for i in range(1,27)]

# The following columns are the infant mortality columns
im_columns = [f'IM_Deaths{i}' for i in range(1,5)]

# Set dtypes to avoid mixed type warnings when importing data
input_dtypes = {'Country': int, 'Year': int, 'Admin1': 'Int64', 'SubDiv': str, 'List': str, 'Cause': str, 'Sex': int,
                'Frmat': str, 'IM_Frmat': str}

# Many of the entries in the mortality columns are blank.  The Int64 (vs int64) dtype allows for NaN values.
input_dtypes.update({col:'Int64' for col in (deaths_columns + im_columns)})

In [3]:
# Load all of the mortality files from icd 7 to 10.
mort_icd7 = pd.read_csv('MortIcd7', dtype=input_dtypes)
mort_icd7['ICD_Version'] = 7

mort_icd8 = pd.read_csv('MortIcd8', dtype=input_dtypes)
mort_icd8['ICD_Version'] = 8

mort_icd9 = pd.read_csv('MortIcd9', dtype=input_dtypes)
mort_icd9['ICD_Version'] = 9

mort_icd10 = pd.concat(
    [pd.read_csv(f'Morticd10_part{i}', dtype=input_dtypes) for i in range(1,6)], axis=0)
mort_icd10['ICD_Version'] = 10

In [4]:
who_mortality = pd.concat([mort_icd7, mort_icd8, mort_icd9, mort_icd10])

# Pandas does not handle NA values in columns used as an index
who_mortality['Admin1'] = who_mortality['Admin1'].fillna(0)
who_mortality['SubDiv'] = who_mortality['SubDiv'].fillna('NONE')

who_mortality.shape

(6068067, 40)

In [5]:
who_mortality.columns

Index(['Country', 'Admin1', 'SubDiv', 'Year', 'List', 'Cause', 'Sex', 'Frmat',
       'IM_Frmat', 'Deaths1', 'Deaths2', 'Deaths3', 'Deaths4', 'Deaths5',
       'Deaths6', 'Deaths7', 'Deaths8', 'Deaths9', 'Deaths10', 'Deaths11',
       'Deaths12', 'Deaths13', 'Deaths14', 'Deaths15', 'Deaths16', 'Deaths17',
       'Deaths18', 'Deaths19', 'Deaths20', 'Deaths21', 'Deaths22', 'Deaths23',
       'Deaths24', 'Deaths25', 'Deaths26', 'IM_Deaths1', 'IM_Deaths2',
       'IM_Deaths3', 'IM_Deaths4', 'ICD_Version'],
      dtype='object')

In [6]:
# See if there are redundant entries accross ICD versions
dup_check_cols = ['Country', 'Year', 'Admin1', 'SubDiv', 'List', 'Cause', 'Sex', 'Frmat', 'IM_Frmat']
who_mortality.duplicated(subset=dup_check_cols).sum()

0

In [7]:
# Since there are no duplicates, this column can now be dropped
who_mortality = who_mortality.drop('ICD_Version', axis=1)

## Consider Age Formats
For some entries, there are multiple formats for deaths and infant deaths (`Frmat` and `IM_Frmat`).  In the WHO documentation, these various formats exist where the reported age did not exactly match default age groups, but are close.  Disregarding `Frmat` will cause some discrepancies when graphing over age groups, such as when Death3 or Pop3 refers to age 1 to 4 instead of age 1, but allows more easy comparisons between countries that use differing age formats.

In [8]:
who_mortality['Frmat'].value_counts()

00    3986330
01    1633169
02     364505
09      19631
03      19065
07      16267
04      14738
08      10422
05       3616
06        324
Name: Frmat, dtype: int64

In [9]:
who_mortality['IM_Frmat'].value_counts()

01    4523881
08    1430501
02      94572
09      19113
Name: IM_Frmat, dtype: int64

In [10]:
non_format_cols = ['Country', 'Year', 'Admin1', 'SubDiv', 'List', 'Cause', 'Sex']
who_mortality.duplicated(subset= non_format_cols).sum()

0

In [11]:
# There are no situations where a cause has multiple reporting formats, so there is no need to combine rows.
who_mortality = who_mortality.drop(['Frmat', 'IM_Frmat'], axis=1)

## Determine all cause mortality
The `Cause` column lists the ICD code for all cause, or for a more specific defined cause of death, which is based on the ICD version defined in `List`.

In [12]:
who_mortality['List'].unique()

array(['07A', '07B', '08A', '08B', '09A', '09B', '09N', '09C', '101',
       '103', '104', '10M', 'UE1'], dtype=object)

In [13]:
# The following is based on the ICD codes from the tables provided in the documentation.
all_cause_code_dict = {
    '07A': 'A000',
    '07B': 'B000',
    '08A': 'A000',
    '08B': 'B000',
    '09A': 'B00',
    '09B': 'B00',
    '09N': 'B00',
    '09C': 'C001',
    '101': '1000',
    '103': 'AAA',
    '104': 'AAA',
    '10M': 'AAA',
    'UE1': 'CH00'
}

In [14]:
all_cause = pd.Series(data=pd.NA, index=who_mortality.index, dtype='boolean')
for icd_list, ac_code in all_cause_code_dict.items():
    who_mask = who_mortality['List']==icd_list
    who_subset = who_mortality[who_mask]
    all_cause_test = (who_subset['Cause'] == ac_code)
    all_cause[who_mask] = all_cause_test
    print(f"{icd_list}: {all_cause_test.sum()} of {who_mask.sum()} where all cause codes.")

assert(all_cause.isna().sum() == 0)

07A: 1640 of 270659 where all cause codes.
07B: 220 of 11090 where all cause codes.
08A: 1758 of 377063 where all cause codes.
08B: 72 of 3564 where all cause codes.
09A: 1190 of 196502 where all cause codes.
09B: 1866 of 638778 where all cause codes.
09N: 496 of 56546 where all cause codes.
09C: 84 of 7644 where all cause codes.
101: 304 of 30099 where all cause codes.
103: 739 of 348113 where all cause codes.
104: 4055 of 4109643 where all cause codes.
10M: 12 of 18114 where all cause codes.
UE1: 4 of 252 where all cause codes.


In [15]:
# The Cause and List columns are no longer needed
who_mortality = who_mortality.drop(['Cause', 'List'], axis=1)

who_mort_all_cause = who_mortality[all_cause]
who_mort_cause_defined = who_mortality[~all_cause]

In [16]:
# Aggregate mortality for defined causes and compare
group_columns = ['Country', 'Year', 'Admin1', 'SubDiv', 'Sex']
who_mort_cause_defined = who_mort_cause_defined.groupby(group_columns).sum()
who_mort_all_cause = who_mort_all_cause.set_index(group_columns)

In [17]:
print(f"All Cause: {who_mort_all_cause.shape}")
print(f"Cause Defined: {who_mort_cause_defined.shape}")

All Cause: (12440, 30)
Cause Defined: (12450, 30)


There appears to be more entries where causes were defined than there are entries where all cause was defined.

In [18]:
intersect_index = who_mort_all_cause.index.intersection(who_mort_cause_defined.index)
difference_index = who_mort_cause_defined.index.difference(who_mort_all_cause.index)
print(f"Intersection Size: {len(intersect_index)}")
print(f"Causes defined, but all cause not reported: {len(difference_index)}")

Intersection Size: 12439
Causes defined, but all cause not reported: 11


In [19]:
compare_eq = (who_mort_all_cause.loc[intersect_index].fillna(0) == who_mort_cause_defined.loc[intersect_index]).sum()
compare_lt = (who_mort_all_cause.loc[intersect_index].fillna(0) < who_mort_cause_defined.loc[intersect_index]).sum()
compare_gt = (who_mort_all_cause.loc[intersect_index].fillna(0) > who_mort_cause_defined.loc[intersect_index]).sum()

pd.DataFrame({
    "All Cause == Defined Cause": compare_eq,
    "All Cause < Defined Cause": compare_lt,
    "All Cause > Defined Cause": compare_gt
             })

,All Cause == Defined Cause,All Cause < Defined Cause,All Cause > Defined Cause
Deaths1,6454,5981,4
Deaths2,6864,5571,4
Deaths3,7115,5322,2
Deaths4,8360,4071,8
Deaths5,8436,3995,8
Deaths6,8472,3959,8
Deaths7,7033,5402,4
Deaths8,7108,5327,4
Deaths9,6946,5489,4
Deaths10,6966,5469,4


While all cause is the same as the sum of defined causes in most cases; this is not always the case.  It is likely that a single death can result in an entry for multiple causes of deaths where defined causes is > all cause.  Where all cause is greater than the sum of defined causes, some deaths may not have had a well defined ICD code.  Using the sum of defined causes of deaths as an approximation for the total deaths will likely not be completely accurate for those 11 entries that do not have an all_cause entry.

In [20]:
who_all_cause_calculated = who_mort_cause_defined.loc[difference_index]
who_all_cause_calculated.shape

(11, 30)

In [21]:
# load country codes to see which countries had all causes calculated.
country_codes = pd.read_csv('country_codes')
country_codes.columns

Index(['country', 'name'], dtype='object')

In [22]:
display_ac_calculated = who_all_cause_calculated.reset_index()
display_ac_calculated['Country'] = display_ac_calculated.merge(country_codes,
                                                              how='left', left_on='Country', right_on='country')['name']
# Deaths1 is total deaths for all ages.
display_ac_calculated[['Country', 'Year', 'Sex', 'Deaths1'] + im_columns] 

,Country,Year,Sex,Deaths1,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
0,Tunisia,2013,9,1,0,0,0,0
1,Barbados,2004,9,1,0,0,0,0
2,Belize,2007,9,3,0,0,0,0
3,Belize,2008,9,1,1,0,0,0
4,Maldives,2010,9,1,1,0,0,0
5,Turkey,2013,9,48,30,11,3,1
6,Turkey,2014,9,73,43,21,3,0
7,Turkey,2015,9,66,32,20,2,1
8,Turkey,2016,9,65,28,22,5,2
9,United Arab Emirates,2019,9,3,2,0,0,0


Sex == 9 refers to undefined sex according to the WHO documentation.

In [23]:
who_mortality = pd.concat([who_mort_all_cause, who_all_cause_calculated])
who_mortality = who_mortality.reset_index()

## Load Population File

In [24]:
pop_columns = [f'Pop{i}' for i in range(1,27)]
pop_dtypes = {'Country': int, 'Year': int, 'Admin1': 'Int64', 'SubDiv': str, 'Sex': int,
                'Frmat': str, 'Lb': 'Int64'}
#pop_dtypes.update({col:'Int64' for col in pop_columns})
# Some population values are given in fractions.

In [25]:
pop_df = pd.read_csv('pop', dtype=pop_dtypes)

In [26]:
# Convert population columns from floats to integers
for col in pop_columns:
    pop_series = pop_df[col]
    pop_df[col] = pop_series.round().astype('Int64')

In [27]:
pop_df.columns

Index(['Country', 'Admin1', 'SubDiv', 'Year', 'Sex', 'Frmat', 'Pop1', 'Pop2',
       'Pop3', 'Pop4', 'Pop5', 'Pop6', 'Pop7', 'Pop8', 'Pop9', 'Pop10',
       'Pop11', 'Pop12', 'Pop13', 'Pop14', 'Pop15', 'Pop16', 'Pop17', 'Pop18',
       'Pop19', 'Pop20', 'Pop21', 'Pop22', 'Pop23', 'Pop24', 'Pop25', 'Pop26',
       'Lb'],
      dtype='object')

In [28]:
# Check for duplicates due to Frmat, as above
non_format_cols = ['Country', 'Year', 'Admin1', 'SubDiv','Sex']
pop_df.duplicated(subset=non_format_cols).sum()

0

In [29]:
pop_df = pop_df.drop('Frmat', axis=1)

In [30]:
# fill NA values Admin1 and SubDiv as above to allow for merge.
pop_df['Admin1'] = pop_df['Admin1'].fillna(0)
pop_df['SubDiv'] = pop_df['SubDiv'].fillna('NONE')

In [31]:
# merge with mortality dataframe.  Use outer join to allow for plotting of population where deaths
# are not reported
print(who_mortality.shape)
who_mortality = who_mortality.merge(pop_df, how='outer', on=['Country', 'Year', 'Admin1', 'SubDiv', 'Sex'])
print(who_mortality.shape)

(12451, 35)
(13431, 62)


In [32]:
who_mortality[['Country', 'Year', 'Admin1', 'SubDiv', 'Sex']].isna().sum()

Country    0
Year       0
Admin1     0
SubDiv     0
Sex        0
dtype: int64

In [33]:
# Replace country code with country name
country_name = who_mortality.merge(country_codes, how='left', left_on='Country', right_on='country')['name']
assert(country_name.isna().sum() == 0)
who_mortality['Country'] = country_name

## Handle Admin1 and SubDiv columns
According to the documentation provided by the WHO, `Admin1` refers to a specified region/Category pertinent to each country and `SubDiv` refers to categories of data.  The documentation indicates that the data refers to the country when both `Admin1` and `SubDiv` are blank (`0` and `NONE` after having NA values filled in).

In [34]:
((who_mortality['Admin1'] != 0) & (who_mortality['SubDiv'] != 'NONE')).sum()

0

There are no cases with non standard `Admin1` and `SubDiv` together; considering each case individually.  Similar to `Cause`, the sum of non-blank entries will be used only when there is not a blank entry available.

In [35]:
who_admin_blank = who_mortality[who_mortality['Admin1']==0]
who_admin_defined = who_mortality[who_mortality['Admin1']!=0]
non_admin_group_cols = ['Country', 'Year', 'SubDiv', 'Sex']
who_admin_blank = who_admin_blank.set_index(non_admin_group_cols)
who_admin_defined = who_admin_defined.groupby(non_admin_group_cols).sum()
who_admin_blank = who_admin_blank.drop('Admin1', axis=1)
who_admin_defined = who_admin_defined.drop('Admin1', axis=1)

In [36]:
admin_intersect_index = who_admin_defined.index.intersection(who_admin_blank.index)
admin_difference_index = who_admin_defined.index.difference(who_admin_blank.index)
print(f"Intersection Size: {len(admin_intersect_index)}")
print(f"Admin1 set, but no corresponding blank record: {len(admin_difference_index)}")

Intersection Size: 77
Admin1 set, but no corresponding blank record: 56


In [37]:
admin_calculated = who_admin_defined.loc[admin_difference_index]
pd.pivot_table(admin_calculated.reset_index(),
               index=['Country', 'Sex'], values=['Year'], aggfunc=['min','max', 'count'])

min   max count
              Year  Year  Year
Country  Sex                  
Israel   1    1950  1974    25
         2    1950  1974    25
Malaysia 1    1977  1979     3
         2    1977  1979     3

In [38]:
who_mortality = pd.concat([who_admin_blank, admin_calculated]).reset_index()
who_mortality.shape

(13311, 61)

Repeat what was done for `Admin1`, but for `SubDiv`.

In [39]:
who_subdiv_blank = who_mortality[who_mortality['SubDiv']=='NONE']
who_subdiv_defined = who_mortality[who_mortality['SubDiv']!='NONE']
non_subdiv_group_cols = ['Country', 'Year', 'Sex']
who_subdiv_blank = who_subdiv_blank.set_index(non_subdiv_group_cols)
who_subdiv_defined = who_subdiv_defined.groupby(non_subdiv_group_cols).sum()
who_subdiv_blank = who_subdiv_blank.drop('SubDiv', axis=1)

In [40]:
subdiv_intersect_index = who_subdiv_defined.index.intersection(who_subdiv_blank.index)
subdiv_difference_index = who_subdiv_defined.index.difference(who_subdiv_blank.index)
print(f"Intersection Size: {len(subdiv_intersect_index)}")
print(f"SubDiv set, but no corresponding blank record: {len(subdiv_difference_index)}")

Intersection Size: 18
SubDiv set, but no corresponding blank record: 200


In [41]:
subdiv_calculated = who_subdiv_defined.loc[subdiv_difference_index]
pd.pivot_table(subdiv_calculated.reset_index(),
               index=['Country', 'Sex'], values=['Year'], aggfunc=['min','max', 'count'])

min   max count
                                    Year  Year  Year
Country                        Sex                  
China                          1    1987  2000    14
                               2    1987  2000    14
Iran (Islamic Republic of)     1    1974  1987    11
                               2    1974  1987    11
Lebanon                        1    2017  2018     2
                               2    2017  2018     2
Malaysia                       1    1997  1997     1
                               2    1997  1997     1
Occupied Palestinian Territory 1    2008  2018    11
                               2    2008  2018    11
Pakistan                       1    1993  1994     2
                               2    1993  1994     2
Paraguay                       1    1961  1991    30
                               2    1961  1991    30
Syrian Arab Republic           1    1977  1977     1
                               2    1977  1977     1
Turkey                         1    1978  2008    28
                               2    1978  2008    28

In [42]:
who_mortality = pd.concat([who_subdiv_blank, subdiv_calculated]).reset_index()
who_mortality.shape

(13237, 60)

In [43]:
who_mortality.duplicated(subset=['Country', 'Year', 'Sex']).sum()

0

## Change sex labels to strings; calculate total for all sex.

In [44]:
sex_series = who_mortality['Sex']
sex_series = sex_series.replace({1:'Male', 2:'Female', 9:'Unspecified'})
sex_series.value_counts()

Male           6439
Female         6439
Unspecified     359
Name: Sex, dtype: int64

In [45]:
who_mortality['Sex'] = sex_series

In [46]:
all_sex = who_mortality.groupby(['Country', 'Year'], as_index=False).sum()
# replace 0 with Na for missing values
all_sex = all_sex.replace(0, np.nan)
all_sex['Sex'] = 'All'
all_sex = all_sex[who_mortality.columns]
who_mortality = pd.concat([who_mortality, all_sex])

In [47]:
who_mortality.shape

(19676, 60)

## Calculate rates

In [48]:
# Calculate total infant mortality
who_mortality['IM_Total'] = who_mortality[im_columns].sum(axis=1).astype('Int64')

In [49]:
rate_columns = [f'Rate{i}' for i in range(1,27)]

for death_col, pop_col, rate_col in zip(deaths_columns+['IM_Total'], pop_columns+['Lb'], rate_columns+['IM_Rate']):
    who_mortality[rate_col] = 1000 * who_mortality[death_col] / who_mortality[pop_col]

In [50]:
# Where population is reported as zero but there is a death count, there are inf values; replace this with NaN
who_mortality[rate_columns+['IM_Rate']] = who_mortality[rate_columns+['IM_Rate']].replace(np.inf, np.nan)

## Create output files

In [51]:
# The color map gets confused by the USA label in Virgin Islands
who_mortality['Country'] = who_mortality['Country'].replace({"Virgin Islands (USA)": "US Virgin Islands"})

In [52]:
index_columns = ['Country', 'Year', 'Sex']
who_mortality = who_mortality.sort_values(index_columns)

In [53]:
deaths_df = who_mortality[index_columns + deaths_columns + ['IM_Total']]
pop_df = who_mortality[index_columns + pop_columns + ['Lb']]
rate_df = who_mortality[index_columns + rate_columns + ['IM_Rate']]

In [54]:
# Column names based on WHO documentation for Deaths/Pop 1 to 26.  'Infant' used for infant deaths, live births, and infant
# mortality rate.
output_columns = ['All Ages', '0 years', '1 year', '2 year', '3 year', '4 year', '5-9 years', '10-14 years',
                  '15-19 years', '20-24 years', '25-29 years', '30-34 years', '35-39 years', '40-44 years',
                  '45-49 years', '50-54 years', '55-59 years', '60-64 years', '65-69 years', '70-74 years',
                  '75-79 years', '80-84 years', '85-89 years', '90-94 years', '95+ years', 'Unspecified', 'Infant']
deaths_df = deaths_df.rename(columns=dict(zip(deaths_columns+['IM_Total'], output_columns)))
pop_df = pop_df.rename(columns=dict(zip(pop_columns + ['Lb'], output_columns)))
rate_df = rate_df.rename(columns=dict(zip(rate_columns + ['IM_Rate'], output_columns)))

In [55]:
deaths_df.to_csv('who_all_cause_deaths.csv', index=False)
pop_df.to_csv('who_population.csv', index=False)
rate_df.to_csv('who_ac_deaths_per_1000.csv', index=False)